In [54]:
from sklearn.datasets import  fetch_california_housing,load_diabetes,fetch_openml
#1461,31,29
x = fetch_openml(data_id=1461, as_frame=True, parser='pandas')


In [55]:
dataset = x['frame']
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Class
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1


In [48]:
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as column_selector
from sklearn.pipeline import FeatureUnion
#from mlxtend.feature_selection import ColumnSelector

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

import numpy as np

from sklearn.neighbors import KNeighborsClassifier

In [63]:
pipeline = Pipeline([
    ('feats', FeatureUnion([
        ('numerical',   
             SimpleImputer(missing_values=np.nan, strategy='mean'),
             column_selector(dtype_include=[np.number])),
        ('categorical', 
             OneHotEncoder(),
             SimpleImputer(missing_values=np.nan, strategy='most_frequent'),
             column_selector(dtype_include=[object, "category"]))
    ])),
    #('clf', KNeighborsClassifier())
])

In [68]:
pipeline = FeatureUnion([
        ('numerical',   
             SimpleImputer(missing_values=np.nan, strategy='mean'),
             column_selector(dtype_include=[np.number])
        ),
        ('categorical', 
             OneHotEncoder(),
             SimpleImputer(missing_values=np.nan, strategy='most_frequent'),
             column_selector(dtype_include=[object, "category"])
        )
    ])


pipeline.fit(dataset[['V1','V3']])

ValueError: too many values to unpack (expected 2)

In [62]:
dataset[['V1','V3']]

,V1,V3
0,58,married
1,44,single
2,33,married
3,47,married
4,33,single
...,...,...
45206,51,married
45207,71,divorced
45208,72,married
45209,57,married


In [87]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

import pandas as pd   
X = pd.DataFrame({
    "documents": ["First item", "second one here", "Is this the last?"],
    "width": [3, 4, 5],
})

ct = ColumnTransformer(
     [
         ("text_preprocess", 
              Pipeline([
                  OneHotEncoder(),
                  SimpleImputer(missing_values='unknown', strategy='most_frequent'),
              ]),
              column_selector(dtype_include=[object, "category"]) 
         ),
         ("num_preprocess", 
              SimpleImputer(missing_values=np.nan, strategy='mean'),
              #MinMaxScaler(),
              column_selector(dtype_include=[np.number])
         )
     ]
)
X_trans = ct.fit_transform(dataset[['V1','V3']]) 

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. 'Pipeline(steps=[OneHotEncoder(),
                SimpleImputer(missing_values='unknown',
                              strategy='most_frequent')])' (type <class 'sklearn.pipeline.Pipeline'>) doesn't.

In [86]:
X_trans

array([[ 0.,  1.,  0., 58.],
       [ 0.,  0.,  1., 44.],
       [ 0.,  1.,  0., 33.],
       ...,
       [ 0.,  1.,  0., 72.],
       [ 0.,  1.,  0., 57.],
       [ 0.,  1.,  0., 37.]])

In [111]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils import check_array

class ReplaceInf(TransformerMixin, BaseEstimator):

    def fit(self, X, y=None):
        # validate and convert if possible:
        X = check_array(X, force_all_finite=False)
        _, counts = np.unique(X, return_counts=True)
        ind = np.argmax(counts)
        self.fill_val_ = X[ind]
        return self

    def transform(self, X):
        X = check_array(X, force_all_finite=False)
        return np.where(X==np.nan, self.fill_val_, X)

In [116]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

def from_bool_to_number(x: np.ndarray) -> np.ndarray:
    return np.where(x, 1.0, 0.0)


def from_number_to_bool(x: np.ndarray) -> np.ndarray:
    return np.where(x > 0.5, True, False)


BooleanTransformer = FunctionTransformer(from_bool_to_number, from_number_to_bool)

# Combine the preprocessing pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    (
        'numerical', 
         Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                #('scaler', StandardScaler())
         ]), 
         column_selector(dtype_include=[np.number]) 
    ),
    (
        'categorical', 
          Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
          ]), 
          column_selector(dtype_include=[object, "category"]) 
    ),
    (
        'boolean', 
          Pipeline([
                ('imputer', ReplaceInf()),
                ('onehot', FunctionTransformer(from_bool_to_number, from_number_to_bool, check_inverse=False))
          ]), 
          column_selector(dtype_include=bool)
    )
    
])

# Add the classifier to the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor ),
    ('classifier', KNeighborsClassifier() )
])

train = data

target = dataset.pop('class')
pipeline.fit(dataset, target)

z = pipeline.predict(dataset, target)




In [117]:
z

array([[58.,  0.,  1.,  0.,  1.,  0.],
       [44.,  0.,  0.,  1.,  1.,  0.],
       [33.,  0.,  1.,  0.,  1.,  0.],
       ...,
       [72.,  0.,  1.,  0.,  1.,  0.],
       [57.,  0.,  1.,  0.,  1.,  0.],
       [37.,  0.,  1.,  0.,  1.,  0.]])

In [119]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone

npl = clone(pipeline)
npl.set_params(classifier=RandomForestClassifier())
npl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F086C03A0>),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklea...
                                                 ('boolean',
                                                  Pipeline(steps=[('imputer',
                                                                   ReplaceInf()),
                                                                  ('onehot',
                                                                   FunctionTransformer(check_inverse=False,
                                                                                       func=<function from_bool_to_number at 0x0000015F07330160>,
                                                                                       inverse_func=<function from_number_to_bool at 0x0000015F086020D0>))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000015F0733E610>)])),
                ('classifier', RandomForestClassifier())])